In [23]:
!pip install apscheduler
!pip install oandapyV20
!pip install oanda_candles
!pip install config


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [19]:
import yfinance as yf
import pandas as pd
# this is primarily test data just to see if our algorithm properly trades
# used in 15 minute intervals to see changes
dataF = pd.read_csv("LME_Official_202406.csv")


0    9985.5
1    9858.5
Name: Cash Ask, dtype: float64

# 1: Import Data using yfinance

# 2 - Define Signal Function
- function that takes in data frame
- when we are using live data, we will always have a data frame and this function takes that in input and returns either a buy or sell signal

In [33]:
# signal generator is used to send signals on whether to buy or sell
def signal_generator(df):
    #essentially basically sets last element in df
    # last close element
    close = df['Cash Ask'].iloc[-1]
    previous_close = df['Cash Ask'].iloc[-2]

    #Focus on Cash Ask
    #sell
    #if cash ask drops by 1%, we want to buy
    if previous_close >= 1.01*close:
        return "Sell"

    #buy
    elif previous_close <= 0.99*close:
        return "Buy"

    else:
        return "Neither"
    #if cash ask increases by 1%, we want to sell
    

#testing of signal_generator code
signal = []
#appends a 0 to start
signal.append(0)
#then loops through data to determine whether the signal generator works
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))

dataF["signal"] = signal
dataF


,datestamp,Cash Bid,Cash Ask,Cash Ask(£),3 Month Bid,3 Month Ask,3 Month Ask(£),DEC 1 Bid,DEC 1 Ask,DEC 2 Bid,DEC 2 Ask,DEC 3 Bid,DEC 3 Ask,signal
0,03/06/2024,9985.00,9985.5,7844.69,10106.0,10107.0,7936.400,10240,10250,10185.0,10195.0,10060,10070,0
1,04/06/2024,9858.00,9858.5,7724.89,9987.0,9989.0,7822.850,10110,10120,10035.0,10045.0,9930,9940,Sell
2,05/06/2024,9764.00,9769.0,7644.57,9889.0,9890.0,7735.020,10015,10025,9960.0,9970.0,9840,9850,Neither
3,06/06/2024,9925.00,9930.0,7774.82,10043.0,10045.0,7861.170,10155,10165,10095.0,10105.0,9935,9945,Buy
4,07/06/2024,9835.00,9840.0,7683.30,9950.0,9955.0,7768.850,10050,10060,9975.0,9985.0,9815,9825,Neither
5,10/06/2024,9694.00,9694.5,7625.06,9829.0,9830.0,7727.380,9945,9955,9865.0,9875.0,9710,9720,Sell
6,11/06/2024,9566.50,9567.0,7510.01,9698.0,9700.0,7610.230,9825,9835,9775.0,9785.0,9615,9625,Sell
7,12/06/2024,9697.00,9697.5,7599.33,9827.0,9828.0,7697.370,9945,9955,9890.0,9900.0,9735,9745,Buy
8,13/06/2024,9732.00,9733.0,7624.76,9859.0,9860.0,7720.010,9965,9975,9880.0,9890.0,9745,9755,Neither
9,14/06/2024,9649.00,9650.0,7596.03,9779.0,9781.0,7694.300,9910,9920,9860.0,9870.0,9750,9760,Neither


In [27]:
dataF.signal.value_counts()

signal
0    13
1     4
2     4
Name: count, dtype: int64

# 3 Connect to Market and execute trades
- establishes connection to market and allows for trading
- in order for this to work, must check to see if broker has API that will allow for us to connect and trade
- Check for JP Morgan API regarding this
- Need to setup OANDA API to test this first (make a demo)
  

In [21]:
# from apscheduler.schedulers.blocking import BlockingScheduler
# from oandapyV20 import API
# import oandapyV20.endpoints.orders as orders
# from oandapyV20.contrib.requests import MarketOrderRequest
# from oanda_candles import Pair, Gran, CandleClient
# from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [35]:
# from config import access_token, accountID
# def get_candles(n):
#     #access_token='XXXXXXX'#you need token here generated from OANDA account
#     client = CandleClient(access_token, real=False)
#     collector = client.get_collector(Pair.EUR_USD, Gran.M15)
#     candles = collector.grab(n)
#     return candles

# candles = get_candles(3)
# for candle in candles:
#     print(float(str(candle.bid.o))>1)

ModuleNotFoundError: No module named 'config'

In [39]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])
    
    

# Executing Orders with a Scheduler

In [41]:
trading_job()

# scheduler = BlockingScheduler()
# scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
# scheduler.start()

NameError: name 'get_candles' is not defined